# Loop 64 Analysis: Final Day Strategy

**Situation:**
- Best LB: 0.0873 (exp_032)
- Target: 0.070180
- Gap: 19.6% improvement needed
- Remaining submissions: 4
- Time: Less than 4 hours

**Goal:** Identify any remaining opportunities to beat the target.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

# Submission history
submissions = [
    ('exp_000', 0.0111, 0.0982),
    ('exp_001', 0.0123, 0.1065),
    ('exp_003', 0.0105, 0.0972),
    ('exp_005', 0.0104, 0.0969),
    ('exp_006', 0.0097, 0.0946),
    ('exp_007', 0.0093, 0.0932),
    ('exp_009', 0.0092, 0.0936),
    ('exp_012', 0.0090, 0.0913),
    ('exp_024', 0.0087, 0.0893),
    ('exp_026', 0.0085, 0.0887),
    ('exp_030', 0.0083, 0.0877),
    ('exp_035', 0.0098, 0.0970),
    ('exp_032', 0.0082, 0.0873),
]

df = pd.DataFrame(submissions, columns=['exp', 'cv', 'lb'])
print(df)
print(f"\nBest LB: {df['lb'].min():.4f} ({df.loc[df['lb'].idxmin(), 'exp']})")
print(f"Best CV: {df['cv'].min():.4f} ({df.loc[df['cv'].idxmin(), 'exp']})")
print(f"Target: 0.070180")

        exp      cv      lb
0   exp_000  0.0111  0.0982
1   exp_001  0.0123  0.1065
2   exp_003  0.0105  0.0972
3   exp_005  0.0104  0.0969
4   exp_006  0.0097  0.0946
5   exp_007  0.0093  0.0932
6   exp_009  0.0092  0.0936
7   exp_012  0.0090  0.0913
8   exp_024  0.0087  0.0893
9   exp_026  0.0085  0.0887
10  exp_030  0.0083  0.0877
11  exp_035  0.0098  0.0970
12  exp_032  0.0082  0.0873

Best LB: 0.0873 (exp_032)
Best CV: 0.0082 (exp_032)
Target: 0.070180


In [2]:
# Linear regression: LB = a*CV + b
slope, intercept, r_value, p_value, std_err = stats.linregress(df['cv'], df['lb'])
print(f"Linear fit: LB = {slope:.2f}*CV + {intercept:.4f}")
print(f"R² = {r_value**2:.4f}")
print(f"\nInterpretation:")
print(f"  - Intercept: {intercept:.4f}")
print(f"  - Target: 0.070180")
print(f"  - Intercept > Target? {intercept > 0.070180}")

# Required CV to hit target
required_cv = (0.070180 - intercept) / slope
print(f"\nRequired CV to hit target: {required_cv:.6f}")
print(f"Best CV achieved: {df['cv'].min():.6f}")
print(f"Improvement needed: {(df['cv'].min() - required_cv) / df['cv'].min() * 100:.1f}%")

Linear fit: LB = 4.36*CV + 0.0520
R² = 0.9566

Interpretation:
  - Intercept: 0.0520
  - Target: 0.070180
  - Intercept > Target? False

Required CV to hit target: 0.004166
Best CV achieved: 0.008200
Improvement needed: 49.2%


In [3]:
# Calculate residuals (LB - predicted LB)
df['predicted_lb'] = slope * df['cv'] + intercept
df['residual'] = df['lb'] - df['predicted_lb']
print("Residual Analysis (LB - Predicted LB):")
print(df[['exp', 'cv', 'lb', 'predicted_lb', 'residual']].sort_values('residual'))
print(f"\nBest residual: {df['residual'].min():.4f} ({df.loc[df['residual'].idxmin(), 'exp']})")
print(f"Worst residual: {df['residual'].max():.4f} ({df.loc[df['residual'].idxmax(), 'exp']})")
print(f"\nMean residual: {df['residual'].mean():.4f}")
print(f"Std residual: {df['residual'].std():.4f}")

Residual Analysis (LB - Predicted LB):
        exp      cv      lb  predicted_lb  residual
0   exp_000  0.0111  0.0982      0.100440 -0.002240
8   exp_024  0.0087  0.0893      0.089967 -0.000667
2   exp_003  0.0105  0.0972      0.097822 -0.000622
10  exp_030  0.0083  0.0877      0.088221 -0.000521
3   exp_005  0.0104  0.0969      0.097385 -0.000485
12  exp_032  0.0082  0.0873      0.087785 -0.000485
9   exp_026  0.0085  0.0887      0.089094 -0.000394
7   exp_012  0.0090  0.0913      0.091276  0.000024
4   exp_006  0.0097  0.0946      0.094331  0.000269
5   exp_007  0.0093  0.0932      0.092585  0.000615
1   exp_001  0.0123  0.1065      0.105677  0.000823
6   exp_009  0.0092  0.0936      0.092149  0.001451
11  exp_035  0.0098  0.0970      0.094767  0.002233

Best residual: -0.0022 (exp_000)
Worst residual: 0.0022 (exp_035)

Mean residual: 0.0000
Std residual: 0.0011


In [4]:
# What if we could achieve the best residual with the best CV?
best_residual = df['residual'].min()
best_cv = df['cv'].min()

optimistic_lb = slope * best_cv + intercept + best_residual
print(f"Optimistic scenario:")
print(f"  Best CV: {best_cv:.4f}")
print(f"  Best residual: {best_residual:.4f}")
print(f"  Optimistic LB: {optimistic_lb:.4f}")
print(f"  Target: 0.070180")
print(f"  Gap: {optimistic_lb - 0.070180:.4f}")
print(f"\nEven with best CV + best residual, we'd be {(optimistic_lb - 0.070180) / 0.070180 * 100:.1f}% away from target")

Optimistic scenario:
  Best CV: 0.0082
  Best residual: -0.0022
  Optimistic LB: 0.0855
  Target: 0.070180
  Gap: 0.0154

Even with best CV + best residual, we'd be 21.9% away from target


In [5]:
# What CV would we need to hit target with best residual?
required_cv_with_best_residual = (0.070180 - intercept - best_residual) / slope
print(f"Required CV (with best residual): {required_cv_with_best_residual:.6f}")
print(f"Best CV achieved: {best_cv:.6f}")
print(f"Improvement needed: {(best_cv - required_cv_with_best_residual) / best_cv * 100:.1f}%")
print(f"\nThis is a {best_cv / required_cv_with_best_residual:.1f}x improvement in CV needed")

Required CV (with best residual): 0.004679
Best CV achieved: 0.008200
Improvement needed: 42.9%

This is a 1.8x improvement in CV needed


In [6]:
# Check if there's any pattern in residuals
print("Residuals by experiment type:")
for _, row in df.sort_values('residual').iterrows():
    print(f"  {row['exp']}: residual = {row['residual']:.4f}, CV = {row['cv']:.4f}, LB = {row['lb']:.4f}")

Residuals by experiment type:
  exp_000: residual = -0.0022, CV = 0.0111, LB = 0.0982
  exp_024: residual = -0.0007, CV = 0.0087, LB = 0.0893
  exp_003: residual = -0.0006, CV = 0.0105, LB = 0.0972
  exp_030: residual = -0.0005, CV = 0.0083, LB = 0.0877
  exp_005: residual = -0.0005, CV = 0.0104, LB = 0.0969
  exp_032: residual = -0.0005, CV = 0.0082, LB = 0.0873
  exp_026: residual = -0.0004, CV = 0.0085, LB = 0.0887
  exp_012: residual = 0.0000, CV = 0.0090, LB = 0.0913
  exp_006: residual = 0.0003, CV = 0.0097, LB = 0.0946
  exp_007: residual = 0.0006, CV = 0.0093, LB = 0.0932
  exp_001: residual = 0.0008, CV = 0.0123, LB = 0.1065
  exp_009: residual = 0.0015, CV = 0.0092, LB = 0.0936
  exp_035: residual = 0.0022, CV = 0.0098, LB = 0.0970


In [7]:
# What experiments have NOT been submitted?
submitted_exps = set(df['exp'].tolist())
print(f"Submitted experiments: {sorted(submitted_exps)}")
print(f"\nTotal experiments: 64")
print(f"Submitted: {len(submitted_exps)}")
print(f"Not submitted: {64 - len(submitted_exps)}")

Submitted experiments: ['exp_000', 'exp_001', 'exp_003', 'exp_005', 'exp_006', 'exp_007', 'exp_009', 'exp_012', 'exp_024', 'exp_026', 'exp_030', 'exp_032', 'exp_035']

Total experiments: 64
Submitted: 13
Not submitted: 51


In [8]:
# Key insight: The CV-LB relationship is very strong (R² = 0.95+)
# The intercept is ~0.052, which is below the target (0.0702)
# This means the target IS theoretically reachable if we can get CV low enough

print("="*60)
print("CRITICAL ANALYSIS")
print("="*60)
print(f"\n1. CV-LB Relationship: LB = {slope:.2f}*CV + {intercept:.4f}")
print(f"   R² = {r_value**2:.4f} (very strong)")
print(f"\n2. Intercept ({intercept:.4f}) < Target ({0.070180:.4f})")
print(f"   This means target IS theoretically reachable!")
print(f"\n3. Required CV to hit target: {required_cv:.6f}")
print(f"   Best CV achieved: {best_cv:.6f}")
print(f"   Gap: {(best_cv - required_cv) / best_cv * 100:.1f}% improvement needed")
print(f"\n4. Best LB achieved: {df['lb'].min():.4f}")
print(f"   Target: 0.070180")
print(f"   Gap: {(df['lb'].min() - 0.070180) / 0.070180 * 100:.1f}% improvement needed")

CRITICAL ANALYSIS

1. CV-LB Relationship: LB = 4.36*CV + 0.0520
   R² = 0.9566 (very strong)

2. Intercept (0.0520) < Target (0.0702)
   This means target IS theoretically reachable!

3. Required CV to hit target: 0.004166
   Best CV achieved: 0.008200
   Gap: 49.2% improvement needed

4. Best LB achieved: 0.0873
   Target: 0.070180
   Gap: 24.4% improvement needed


In [9]:
# What approaches haven't been fully explored?
print("="*60)
print("UNEXPLORED APPROACHES")
print("="*60)
print("""
1. Graph Neural Networks (GNN)
   - The paper's GNN achieved MSE 0.0039
   - This would translate to LB ~0.07 (close to target!)
   - But implementation is complex and risky with limited time

2. Ensemble of ALL best models
   - We have exp_032 (best LB), exp_030 (second best LB)
   - Could try averaging their predictions

3. Different ensemble weights
   - exp_032 uses GP(0.15) + MLP(0.55) + LGBM(0.30)
   - Could try GP(0.10) + MLP(0.60) + LGBM(0.30)

4. Post-processing / calibration
   - Adjust predictions based on known patterns

5. Different feature combinations
   - We've tried many, but there might be unexplored combinations
""")
print(f"\nRemaining submissions: 4")
print(f"Time: Less than 4 hours")
print(f"\nRECOMMENDATION: Focus on high-impact, quick experiments")

UNEXPLORED APPROACHES

1. Graph Neural Networks (GNN)
   - The paper's GNN achieved MSE 0.0039
   - This would translate to LB ~0.07 (close to target!)
   - But implementation is complex and risky with limited time

2. Ensemble of ALL best models
   - We have exp_032 (best LB), exp_030 (second best LB)
   - Could try averaging their predictions

3. Different ensemble weights
   - exp_032 uses GP(0.15) + MLP(0.55) + LGBM(0.30)
   - Could try GP(0.10) + MLP(0.60) + LGBM(0.30)

4. Post-processing / calibration
   - Adjust predictions based on known patterns

5. Different feature combinations
   - We've tried many, but there might be unexplored combinations


Remaining submissions: 4
Time: Less than 4 hours

RECOMMENDATION: Focus on high-impact, quick experiments
